In [ ]:
# ─────────────────────────────────────────
# 필요 패키지 설치 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def go_to_next_page(driver, wait):
    try:
        curr = int(driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip())
    except:
        return False
    nxt = curr + 1
    # 같은 그룹 내 번호 클릭
    try:
        wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{nxt}"]')
        )).click()
    except TimeoutException:
        # '다음 10 페이지' 클릭 후 재시도
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next"))).click()
            wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{nxt}"]')
            )).click()
        except TimeoutException:
            return False
    # 실제로 페이지 바뀔 때까지 대기
    try:
        wait.until(lambda d: d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip() == str(nxt))
        return True
    except TimeoutException:
        return False

def collect_reviews(driver, wait):
    reviews = []
    # 리뷰 탭 클릭
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
        time.sleep(1)
    except TimeoutException:
        return reviews

    # 페이지 순회
    while True:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))
        for el in driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner"):
            txt = el.text.strip()
            if txt:
                reviews.append(txt)
        if not go_to_next_page(driver, wait):
            break
    return reviews

def crawl_and_save(driver, wait, name):
    """
    현재 로드된 상품 리스트에 대해,
    모든 상품 페이지를 열어 리뷰 수집 → 리스트 반환 후 파일로 저장
    """
    main_handle = driver.current_window_handle
    links = [a.get_attribute("href")
             for a in driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")]

    all_rev = []
    for href in links:
        driver.execute_script("window.open(arguments[0], '_blank');", href)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(1)
        all_rev += collect_reviews(driver, wait)
        driver.close()
        driver.switch_to.window(main_handle)

    # 저장
    df = pd.DataFrame(all_rev, columns=["리뷰내용"])
    csv_name = f"{name}_reviews.csv"
    xlsx_name = f"{name}_reviews.xlsx"
    df.to_csv(csv_name, index=False, encoding="utf-8-sig")
    df.to_excel(xlsx_name, index=False)
    print(f"✅ [{name}] 리뷰 {len(all_rev)}개 저장 → {csv_name}, {xlsx_name}")

def main():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)  # 창 닫히지 않도록
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    wait = WebDriverWait(driver, 10)

    # 1) 올리브영 메인 페이지 접속 & 카테고리 열기
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")
    wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()
    time.sleep(1)

    # ──────────── 스킨/토너 리뷰 ────────────
    wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '스킨/토너')]"))).click()
    time.sleep(1)
    crawl_and_save(driver, wait, "skin_toner")  # 파일명 prefix: skin_toner

    # ───────── 에센스/세럼/앰플 리뷰 ─────────
    wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        "a[data-attr*='에센스/세럼/앰플']"
    ))).click()
    time.sleep(1)
    crawl_and_save(driver, wait, "essence_serum_ampoule")  # essence_serum_ampoule_reviews.*

    # ───────────── 크림 리뷰 ─────────────
    wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//ul[@class='loc_history']//a[@class='cate_y' and normalize-space(text())='크림']"
    ))).click()
    time.sleep(1)
    crawl_and_save(driver, wait, "cream")

    # ─────────── 아이크림 리뷰 ────────────
    wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//ul[@class='cate_list_box']//a[normalize-space(text())='아이크림']"
    ))).click()
    time.sleep(1)
    crawl_and_save(driver, wait, "eyecream")

    driver.quit()

if __name__ == "__main__":
    main()
